In [3]:
using CSV
using DataFrames
using Statistics
using MultivariateStats
using Distances
using LinearAlgebra

In [13]:
versioninfo()

Julia Version 1.5.3
Commit 788b2c77c1 (2020-11-09 13:37 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Xeon(R) CPU E3-1225 v5 @ 3.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake)
Environment:
  JULIA_HOME = "C:\Users\KANKI\AppData\Local\Programs\Julia 1.5.3\bin"
  JULIA_PKGDIR = C:\Users\KANKI\AppData\Local\Julia-1.0.2\Pkg


Tutorial1と同じディレクトリ内であれば，すぐに実行できます．  
ただし，上記のライブラリをインストールする必要があります．(Document->Juliaのライブラリの入れ方)  
JuliaをJupyterで使えるようにする方法は以下を参照．  

In [10]:
path_file_face="sample/sample_output/modelA01/ply_parts/faces_dataset_mesh.csv"
path_file_v5="sample/sample_output/modelA01/ply_parts/vertice_dataset_mesh.csv"
path_file_v1="sample/sample_output/modelA01/ply_parts/vertice_dataset_vert.csv"
path_file_n5="sample/sample_output/modelA01/ply_parts/vertice_normal_vert.csv"

table_f=CSV.read(path_file_face,header=false, DataFrame)
table_v5=CSV.read(path_file_v5,header=false, DataFrame)
table_v1=CSV.read(path_file_v1,header=false, DataFrame)
table_n5=CSV.read(path_file_n5,header=false, DataFrame)

table_f=convert(Matrix,table_f)
table_v5=convert(Matrix,table_v5)
table_v1=convert(Matrix,table_v1)
table_n5=convert(Matrix, table_n5)
row=size(table_f)[1]

function calc_terrain_variables(table_f, table_v5, table_v1, kernels, iter)
    row, col=size(table_f)
    row_v1, col_v1=size(table_v1)
    metav=projection(fit(PCA,table_v1'))
    tilt_direction=180+sign(metav[2])*(90+90-acos(metav[1]/(metav[1]^2+metav[2]^2)^0.5)/pi*180)
    g=zeros(Float32,row_v1,3)
    ng=zeros(Float32,row_v1,3)
    terrain_variables=zeros(Float32, row, 2+size(kernels)[1]*8)

    bottom_all=findmin(table_v5[:,3])[1]    
    for c=1:row
        for d=1:3
            g[c,d]=(table_v5[table_f[c,1]+1,d]+table_v5[table_f[c,2]+1,d]+table_v5[table_f[c,3]+1,d])/3
            ng[c,d]=(table_n5[table_f[c,1]+1,d]+table_n5[table_f[c,2]+1,d]+table_n5[table_f[c,3]+1,d])/3
        end
    end

    for c=1:row
        terrain_variables[c,1]=g[c,3]
        terrain_variables[c,2]=g[c,3]-bottom_all
    end
    
    kernels=kernels.^2
    st_index=ones(Int, 9)*2+Vector(0:8)*size(kernels)[1]
    
    for c=1:iter
        distance=zeros(Float32, row_v1, 2)
        for i2=1:row_v1
            distance[i2,1]=(table_v1[i2,1]-g[c,1])^2+(table_v1[i2,2]-g[c,2])^2
            distance[i2,2]=distance[i2,1]+(table_v1[i2,3]-g[c,3])^2
        end
        
        
        for (countk, kernel) in enumerate(kernels)
            index_p=findall(distance[:,1] .< kernel)
            ver_in_ker_p=zeros(Float32, size(index_p)[1], 1)
            for (i, index) in enumerate(index_p)
                ver_in_ker_p[i,1]=table_v1[index[1],3]
            end 
            
            index_v=findall(distance[:,2] .< kernel)
            ver_in_ker=zeros(Float32, size(index_v)[1], 3)
            for (i, index) in enumerate(index_v)
                ver_in_ker[i,1:3]=table_v1[index[1],1:3]
            end
            
            bottom=findmin(ver_in_ker_p[:,1])[1]
            meanz=Statistics.mean(ver_in_ker_p[:,1])
            #Calculating rugosity and bpi
            terrain_variables[c,countk+st_index[1]]=g[c,3]-bottom
            terrain_variables[c,countk+st_index[2]]=g[c,3]-meanz
            M=fit(PCA,ver_in_ker')

            
            if size(projection(M))[2]==2
                v3=cross(projection(M)[:,1],projection(M)[:,2])
            elseif size(projection(M))[2]==3
                v3=projection(M)[:,3]
            else
                println(size(projection(M)))
            end
            v3=v3*sign(ng[c,3])*sign(v3[3])
            terrain_variables[c,countk+st_index[3]]=90-acos(v3[1]/(v3[1]^2+v3[2]^2)^0.5)/pi*180
            terrain_variables[c,countk+st_index[4]]=90-acos(v3[2]/(v3[1]^2+v3[2]^2)^0.5)/pi*180
            terrain_variables[c,countk+st_index[5]]=180+sign(v3[2])*(90+terrain_variables[c,countk+10])
            terrain_variables[c,countk+st_index[6]]=findmin([abs(tilt_direction-terrain_variables[c,countk+18]),
                    360-abs(tilt_direction-terrain_variables[c,countk+18])])[1]
            terrain_variables[c,countk+st_index[7]]=((pi/2)-atan(v3[3]/((v3[1]^2+v3[2]^2)^0.5)))/pi*180

            distances=zeros(Float32, size(index_v)[1], 1)
            d=-dot(v3, [mean(ver_in_ker[:,1]), mean(ver_in_ker[:,2]), mean(ver_in_ker[:,3])])
            for j in 1:size(index_v)[1]
                distance[j,1]=(dot(v3, ver_in_ker[j,1:3])+d)/sqrt(dot(v3,v3))
            end
            terrain_variables[c,countk+st_index[8]]=mean(distance)
        end
    end
    return terrain_variables
end

kernels=[0.1, 0.2, 0.4, 0.8]
@time terrain_variables=calc_terrain_variables(table_f, table_v5, table_v1, kernels, 100)

  2.364810 seconds (7.63 M allocations: 2.114 GiB, 12.93% gc time)


23077×34 Array{Float32,2}:
 -1.17563   0.756259  0.645916   …  4.17202  4.16292  4.12406  3.97957
 -0.746547  1.18534   0.0737836     4.90251  4.89325  4.86727  4.74132
 -0.755625  1.17627   0.0725781     4.87502  4.86615  4.84067  4.71473
 -0.738991  1.1929    0.0575961     4.97854  4.96933  4.94168  4.81966
 -0.758527  1.17336   0.0733508     4.84653  4.83708  4.81173  4.68333
 -0.762139  1.16975   0.0762982  …  4.81954  4.80958  4.7837   4.65352
 -0.752011  1.17988   0.0763152     4.89928  4.89102  4.86527  4.74157
 -0.722222  1.20967   0.0595359     5.07276  5.06478  5.03602  4.91963
 -0.741074  1.19082   0.0547494     4.96863  4.95852  4.92998  4.80658
 -0.752242  1.17965   0.0682678     4.86633  4.85581  4.82795  4.69929
 -0.754886  1.177     0.0762733  …  4.84004  4.82935  4.80076  4.67095
 -0.777839  1.15405   0.0812641     4.75223  4.74224  4.71641  4.58402
 -0.758571  1.17332   0.073163      4.84119  4.83301  4.80851  4.68297
  ⋮                              ⋱  ⋮             